In [1]:
import os

In [2]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [3]:
%pwd

'd:\\project-ruang-guru\\Debugging\\Code\\Custom-Debugging-Code\\research'

In [4]:
!mkdir test_repo

A subdirectory or file test_repo already exists.


In [5]:
repo_path = "D:\\project-ruang-guru\\Project_DS_Intern\\source\\rg-ds-chatbot-api\\ds_chatbot\\api"

# Repo.clone_from("https://github.com/entbappy/End-to-end-ML-Project-Implementation", to_path=repo_path)

In [6]:
loader = GenericLoader.from_filesystem(repo_path,
                                        glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [7]:
documents = loader.load()

In [8]:
documents

[Document(page_content='import json\nimport logging\n\nimport urllib3\nfrom fastapi import Header\nfrom ds_chatbot.config import AUTH_CONFIG\nfrom dataclasses import dataclass\nfrom ds_chatbot.config import SECRET_CONFIG\n\n\nlogging.captureWarnings(True)\nlogger = logging.getLogger(__name__)\n\nhttp = urllib3.HTTPConnectionPool(\n    AUTH_CONFIG.HOST,\n    maxsize=AUTH_CONFIG.HTTP_CONNECTION_POOL_SIZE,\n    headers=urllib3.make_headers(keep_alive=True, accept_encoding=True),\n)\n\n\nclass AuthException(Exception):\n    def __init__(self, message: str):\n        self.message = message\n\n\n@dataclass\nclass Context:\n    user_role: str\n    user_code: str\n\n\nclass Authenticate:\n    def __init__(self, allowed_roles: list = None):\n        self.allowed_roles = allowed_roles\n\n    def __call__(self, authorization: str = Header(None), user_role: str = Header(None), user_code: str = Header(None), internal_api_password: str = Header(None)):\n        if AUTH_CONFIG.AUTHENTICATION_ACTIVE i

### Chunkings

In [9]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 2000,
                                                             chunk_overlap = 200)

In [10]:
texts = documents_splitter.split_documents(documents)

In [11]:
len(texts)

36

In [12]:
import os

### Embedding model

In [13]:
os.environ["OPENAI_API_KEY"] = "********"

In [14]:
embeddings=OpenAIEmbeddings(disallowed_special=())

### Knowledge base (vector DB)

In [15]:
# vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./data-gpt-3.5-1')
# vectordb.persist()

In [ ]:
vectordb = Chroma.from_documents(
    documents=texts,
    collection_name="rag-chroma",
    embedding=embeddings,
)

### LLM Wrapper

In [16]:
# llm = ChatOpenAI(model_name="gpt-4")
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

In [17]:
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

In [18]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":5}), memory=memory)

### Q&A

In [19]:
question = "Apakah ada bug pada kode yang diberikan"

In [20]:
result = qa(question)
print(result['answer'])

Tidak, berdasarkan potongan kode yang diberikan, tidak terlihat adanya bug yang jelas.


In [21]:
question = "Apakah ada kode yang perlu di optimisasi"

In [22]:
result = qa(question)
print(result['answer'])

Kode yang diberikan terlihat sudah cukup baik dan tidak memerlukan optimisasi yang signifikan. Kode tersebut sudah terstruktur dengan baik dan mudah dibaca. Jika ada bagian tertentu yang ingin dioptimalkan, itu mungkin lebih pada preferensi atau kebutuhan spesifik proyek yang bersangkutan.


In [23]:
from pprint import pprint
pprint(result['answer'])

('Kode yang diberikan terlihat sudah cukup baik dan tidak memerlukan '
 'optimisasi yang signifikan. Kode tersebut sudah terstruktur dengan baik dan '
 'mudah dibaca. Jika ada bagian tertentu yang ingin dioptimalkan, itu mungkin '
 'lebih pada preferensi atau kebutuhan spesifik proyek yang bersangkutan.')
